In [ ]:
!pip install torch wandb accelerate peft wandb trl bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 17.2 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import os
import transformers
import torch
from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM,Trainer,TrainingArguments
from transformers import BitsAndBytesConfig, GemmaTokenizer

In [ ]:
model_id='google/gemma-2b'
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
from google.colab import userdata
os.environ['HF_TOKEN']=userdata.get('HF_TOKEN')

In [ ]:
tokenizers=AutoTokenizer.from_pretrained(model_id)
model=AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    token=os.environ['HF_TOKEN']
    )

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
lora_config=LoraConfig(
    r=8,
    target_modules=['q_proj','o_proj','k_proj','v_proj'
    'gate_proj','up_proj','down_proj'
    ],
    task_type="CAUSAL_LM"
)

In [ ]:
dataset=load_dataset('Abirate/english_quotes')
dataset

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags'],
        num_rows: 2508
    })
})

In [ ]:
from peft import get_peft_model
model=get_peft_model(model,lora_config)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'google/gemma-2b' to 'None'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
sample=dataset['train'].select(range(1))
sample

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 1
})

In [ ]:
def format_for_training(example):
    prompt = (
        "<bos>### Instruction:\n"
        "You are an AI that identifies the author of a given quote. "
        "Output only the author's name, nothing else.\n\n"
        "### Quote:\n"
        f"{example['quote']}\n\n"
        "### Author:\n"
    )
    completion = f"{example['author']}<eos>"
    return {'text': prompt + completion}
sample=sample.map(format_for_training)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
dataset=dataset['train'].map(format_for_training)
dataset

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Dataset({
    features: ['quote', 'author', 'tags', 'text'],
    num_rows: 2508
})

In [ ]:
dataset['train'][0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 'text': "<bos>### Instruction:\nYou are an AI that identifies the author of a given quote. Output only the author's name, nothing else.\n\n### Quote:\n“Be yourself; everyone else is already taken.”\n\n### Author:\nOscar Wilde<eos>"}

In [ ]:
def tokenization(example):
  return tokenizers(example['text'],truncation=True,max_length=512)

In [ ]:
tokenizers.pad_token=tokenizers.eos_token
tokenizers.pad_token

'<eos>'

In [ ]:
tokenized=dataset.map(tokenization,batched=True)
tokenized

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

Dataset({
    features: ['quote', 'author', 'tags', 'text', 'input_ids', 'attention_mask'],
    num_rows: 2508
})

In [ ]:
tokenized=tokenized.remove_columns(['quote', 'author', 'tags', 'text'])
tokenized

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2508
})

In [ ]:
args=TrainingArguments(
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    weight_decay=0.001,
    optim="paged_adamw_8bit"
)

In [ ]:
trainer=SFTTrainer(
    model=model,
    args=args,
    train_dataset=tokenized
)
trainer.train()

Truncating train dataset:   0%|          | 0/2508 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: gajasankarraja (gajasankarraja-karpagam-college-of-engineering) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.222900
200,1.059300
300,0.968200
400,0.936700


TrainOutput(global_step=471, training_loss=1.0307037450705365, metrics={'train_runtime': 1215.3118, 'train_samples_per_second': 6.191, 'train_steps_per_second': 0.388, 'total_flos': 1.1556551347224576e+16, 'train_loss': 1.0307037450705365, 'entropy': 0.9110107423981171, 'num_tokens': 629982.0, 'mean_token_accuracy': 0.7984375142798407, 'epoch': 3.0})

In [ ]:
import torch

def infer_author(quote, model, tokenizer):
    prompt = (
        "<bos>### Instruction:\n"
        "You are an AI that identifies the author of a given quote. "
        "Output only the author's name, nothing else.\n\n"
        "### Quote:\n"
        f"{quote}\n\n"
        "### Author:\n"
    )
    inputs = tokenizers(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            do_sample=False,
            temperature=0.0,
            pad_token_id=tokenizers.eos_token_id
        )

    generated_text = tokenizers.decode(outputs[0], skip_special_tokens=True)
    author = generated_text.split("### Author:\n")[-1].strip()
    return author
input_quote = "Without music, life would be a mistake."
result = infer_author(input_quote, model, tokenizers)
print(f"Quote: '{input_quote}'")
print(f"Author: '{result}'")

Quote: 'Without music, life would be a mistake.'
Author: 'Friedrich Nietzsche'


In [ ]:
trainer.push_to_hub()

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ner_output/tokenizer.json:  19%|#8        | 6.46MB / 34.4MB            

  ...er_output/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...adapter_model.safetensors:   2%|2         |  549kB / 27.3MB            

  ...587948.89cb7c15f94a.797.0:   2%|2         |   161B / 8.04kB            

  ..._output/training_args.bin:   2%|2         |   125B / 6.22kB            

CommitInfo(commit_url='https://huggingface.co/gajasankarraja/trainer_output/commit/e534e431504d743be42afb6462d57f50c52e9e1c', commit_message='End of training', commit_description='', oid='e534e431504d743be42afb6462d57f50c52e9e1c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gajasankarraja/trainer_output', endpoint='https://huggingface.co', repo_type='model', repo_id='gajasankarraja/trainer_output'), pr_revision=None, pr_num=None)